# Part 3: Joining Data

## Imports

In [59]:
import pandas as pd
import sqlite3

from utils import export

## Join Part 1 and Part 2 data to create a new dataframe

In [2]:
dirname = '../data/'
stations_df = pd.read_csv(dirname + 'stations_clean.csv', sep=',', index_col=False)
yelp_places_df = pd.read_csv(dirname + 'yelp_places_clean.csv', sep=',', index_col=False)
yelp_categories_df = pd.read_csv(dirname + 'yelp_categories_clean.csv', sep=',', index_col=False)

In [4]:
stations_df.sample(5)

,station_id,name,latitude,longitude,altitude,slots,free_bikes,empty_slots
69,90352566ee5193e528aeadc8f07619f1,LC014 - Parque Araucano,-33.403727,-70.573429,690.0,17,5,11
124,bde0622e2e2c885c602ddc359b7115ee,LC036 - Las Condes / Valle Alegre,-33.378284,-70.528372,788.0,11,6,5
1,3983dd515589a80338dd44a28f5ec414,V34 - Mestizo,-33.394000,-70.600000,655.0,23,8,15
137,45d51e231d79a803aa7b2d1a41d1fc3f,P45 - Eliodoro Yañez / Holanda,-33.429568,-70.600625,620.0,14,2,11
15,183c0afbd2b3824204f4433b4fe7e2ac,V06 - Vespucio / Vitacura,-33.399090,-70.588239,680.0,14,9,5


In [5]:
yelp_places_df.sample(5)

,yelp_id,name,review_count,price,rating,distance,station_id
8665,DO8t_u9A0sWMg3Lt_p9WYA,La Casa en el Aire,8,2.0,3.9,233.624118,7107177993db3b28510d1c549988933e
6590,cFBSISEk7mkAekUsNVp-1Q,Tostaduría Maravilla,3,NaN,4.3,27.879315,cf82a49da234dca3281e8dabbf84a949
4566,6MKqYt9Ab6yN7q9qz1ppfA,Tiare Restaurant,2,NaN,3.5,1180.269868,9c00c9be4009c8b5a4f324ef558239a7
9569,LGwEqYL0reUsRdtqE44ITQ,Chipe Libre - República Independiente del Pisco,80,3.0,4.5,867.148232,203d5ff11fe62fefdd74920ddd431c15
7936,PS7Y3zRGs4xh1GGyZKm_Nw,Elkika Ilmenau Tobalaba,21,2.0,3.6,510.425759,ca71578b9ff5c3086bb1683947a9d557


In [6]:
yelp_categories_df.sample(5)

,yelp_id,category_name
1745,nFnsqiq9UVhACMHHFGeL5w,cafes
2,8N6Y3HsLXHBeFwhlO0YxwQ,cocktail bars
1423,0KbH2csVBJ5Pfm8fbRikyw,ice cream & frozen yogurt
1261,Jg8K0xZqSV3LCgswX0dX5A,empanadas
2189,wuU-3z2I46dKlI51xeXAwA,kids activities


Are there any station_ids that are in `stations_df` but not in `yelp_places_df`? (Spoiler: there shouldn't be!)

In [21]:
station_ids = set(stations_df['station_id'].to_list())
yelp_station_ids = set(yelp_places_df['station_id'].to_list())
station_ids.issubset(yelp_station_ids), yelp_station_ids.issubset(station_ids)

(True, True)

The sets of station_ids in `stations_df` and `yelp_places_df` are equal. Therefore, I'll (inner) merge (*i.e.* SQL join) these two tables on `station_id`. But, first, I'll rename a pair of columns to avoid future confusion.

In [33]:
stations_df.rename({'name': 'station_name'}, axis='columns', inplace=True)
yelp_places_df.rename({'name': 'place_name'}, axis='columns', inplace=True)

In [34]:
stations_places_df = stations_df.merge(yelp_places_df, how='inner', on='station_id')

In [35]:
stations_places_df.sample(5)

,station_id,station_name,latitude,longitude,altitude,slots,free_bikes,empty_slots,yelp_id,place_name,review_count,price,rating,distance
1852,c393f4dec0bacaf5f06dfbac66cfdba0,P38 - Québec,-33.436129,-70.625881,592.0,15,4,10,7-68GEE37qW-_rGS2tldzQ,Siete Negronis,9,3.0,4.4,824.683136
8653,7107177993db3b28510d1c549988933e,P21 - Patio Bellavista,-33.434170,-70.634461,587.0,23,1,20,ZKpwNAFay09AiBHPX7jdNw,Original Green Roasters,24,2.0,4.5,825.542787
3372,90352566ee5193e528aeadc8f07619f1,LC014 - Parque Araucano,-33.403727,-70.573429,690.0,17,5,11,Ued5I7gMq6K448roG3-vWw,Emporio Armani Caffe,5,NaN,3.8,522.574415
8471,eefd9aa3f553ab4d75d52f68a5ff90d4,N13 - Ricardo Lyon / Hernan Cortes,-33.446733,-70.602506,600.0,15,5,10,_8tBXpRH7wpyiOwml4QNIg,Bar Valdivia,3,NaN,4.3,956.644486
148,405a3a5ca08c7536d3eb286cf8553025,V10 - Casa Costanera,-33.398000,-70.598000,653.0,11,8,3,mfwL_53Wdl4212bEWJsu_A,El Lobby Millesime,1,NaN,5.0,635.712939


Next, I could merge `stations_places_df` with `yelp_categories_df` on `yelp_id`. The column `station_id` will not longer serve as a primary key however.

Are there any yelp_ids that are in `stations_places_df` that are not in `yelp_categories_df`? It's certainly possible that there are businesses returned by a Yelp search that don't have any associated categories.

In [48]:
yelp_ids_1 = set(stations_places_df['yelp_id'].to_list())
yelp_ids_2 = set(yelp_categories_df['yelp_id'].to_list())
print(f'num yelp IDs: {len(yelp_ids_1)}, {len(yelp_ids_2)}')
print(f'yelp_ids_2 contained in yelp_ids_1: {yelp_ids_2.issubset(yelp_ids_1)}')
yelp_id_diff = yelp_ids_1 - yelp_ids_2
print('yelp IDs in yelp_id_1 but not in yelp_id_2:')
for id in yelp_id_diff:
    print(f'    {id}')

num yelp IDs: 1756, 1748
yelp_ids_2 contained in yelp_ids_1: True
yelp IDs in yelp_id_1 but not in yelp_id_2:
    JX_Axp2MRPwmDhHUw0hZqw
    S3qk-oT76pT6G_VA4Y4j0w
    pLh6ImLqAWPWNICWUz70Uw
    6XnJsC8OZyLuRoWN9wRmEA
    JcJrUd2jyEGUCYU99QtkUA
    g79bELS1rZheFdxkXmzKtw
    B45kBL6rxN7G2udwz2epQw
    05SHJ9668PZQXIhMjp9ACA


To ensure that I keep all businesses in the final merged DataFrame, I'll left merge stations_places_df to yelp_categories_df.  

In [49]:
stations_places_cats_df =\
    stations_places_df.merge(yelp_categories_df, how='left', on='yelp_id')

In [50]:
stations_places_cats_df.head()

,station_id,station_name,latitude,longitude,altitude,slots,free_bikes,empty_slots,yelp_id,place_name,review_count,price,rating,distance,category_name
0,e1593acef03a0fd770595370586bc358,P31 - Estación Canal 13,-33.428334,-70.627312,599.0,13,6,7,8N6Y3HsLXHBeFwhlO0YxwQ,Barrica 94,78,2.0,4.3,988.631270,wine bars
1,e1593acef03a0fd770595370586bc358,P31 - Estación Canal 13,-33.428334,-70.627312,599.0,13,6,7,8N6Y3HsLXHBeFwhlO0YxwQ,Barrica 94,78,2.0,4.3,988.631270,chilean
2,e1593acef03a0fd770595370586bc358,P31 - Estación Canal 13,-33.428334,-70.627312,599.0,13,6,7,8N6Y3HsLXHBeFwhlO0YxwQ,Barrica 94,78,2.0,4.3,988.631270,cocktail bars
3,e1593acef03a0fd770595370586bc358,P31 - Estación Canal 13,-33.428334,-70.627312,599.0,13,6,7,U15KdCy6HH05WISNtN9plQ,Restaurant Peumayén,73,4.0,4.5,856.161401,latin american
4,e1593acef03a0fd770595370586bc358,P31 - Estación Canal 13,-33.428334,-70.627312,599.0,13,6,7,y2FR-JBGie4wiW6RjtKLhg,Aquí Está Coco,58,4.0,4.3,1099.691315,seafood


In [55]:
print(f'shape: {stations_places_cats_df.shape}')
print(f'columns:\n {stations_places_cats_df.columns}')
print(f'dtypes:\n {stations_places_cats_df.dtypes}')

shape: (15212, 15)
columns:
 Index(['station_id', 'station_name', 'latitude', 'longitude', 'altitude',
       'slots', 'free_bikes', 'empty_slots', 'yelp_id', 'place_name',
       'review_count', 'price', 'rating', 'distance', 'category_name'],
      dtype='object')
dtypes:
 station_id        object
station_name      object
latitude         float64
longitude        float64
altitude         float64
slots              int64
free_bikes         int64
empty_slots        int64
yelp_id           object
place_name        object
review_count       int64
price            float64
rating           float64
distance         float64
category_name     object
dtype: object


In [58]:
export(stations_places_cats_df, 'stations_places_cats.csv')

Provide a visualization that you used as part of your EDA process. Explain the initial pattern or relationship you discoved through this visualization. 

# Database

### Put results in an SQLite database

In [67]:
# Define a SQLite connection and cursor.
# Note that dirname was defined above.
con = sqlite3.connect(dirname + 'stations_places_cats.db')
cur = con.cursor()

In [65]:
stations_places_cats_df.dtypes

station_id        object
station_name      object
latitude         float64
longitude        float64
altitude         float64
slots              int64
free_bikes         int64
empty_slots        int64
yelp_id           object
place_name        object
review_count       int64
price            float64
rating           float64
distance         float64
category_name     object
dtype: object

In [116]:
# Define the SQLite table creation command before executing it.
sql_create_cmd = r"""
    CREATE TABLE stations_places_cats (
        station_id TEXT,
        station_name TEXT,
        latitude REAL,
        longitude REAL,
        altitude REAL,
        slots INTEGER,
        free_bikes INTEGER,
        empty_slots INTEGER,
        yelp_id TEXT,
        place_name TEXT,
        review_count INTEGER,
        price REAL,
        rating REAL,
        distance REAL,
        category_name TEXT
    );
"""
print(sql_create_cmd)


    CREATE TABLE stations_places_cats (
        station_id TEXT,
        station_name TEXT,
        latitude REAL,
        longitude REAL,
        altitude REAL,
        slots INTEGER,
        free_bikes INTEGER,
        empty_slots INTEGER,
        yelp_id TEXT,
        place_name TEXT,
        review_count INTEGER,
        price REAL,
        rating REAL,
        distance REAL,
        category_name TEXT
    );



In [146]:
cur.execute('DROP TABLE IF EXISTS stations_places_cats')
cur.execute(sql_create_cmd)

In [147]:
res = cur.execute('SELECT name FROM sqlite_master')
res.fetchone()

('stations_places_cats',)

I've successfully created a database. But it's empty. To fill it:

In [148]:
def sql_insert_cmd(df: pd.DataFrame, db: str, row: int) -> str:
    """
    Return the SQL command that will insert a DataFrame row into the database.
    """
    row_list = df.loc[row, :].to_list()
    row_str = str(row_list).replace('nan', 'NULL').replace('[', '(').replace(']', ')')
    return 'INSERT INTO ' + db + ' VALUES ' + row_str

In [149]:
for row in range(stations_places_cats_df.shape[0]):
    sql_cmd = sql_insert_cmd(stations_places_cats_df, 'stations_places_cats', row)
    cur.execute(sql_cmd)

In [150]:
con.commit()

I'll verify that inserting into the database woked as expected.

In [151]:
res = cur.execute('SELECT * FROM stations_places_cats')
records = res.fetchall()
len(records), len(records[0])

(15212, 15)

Phew!

Look at the data before and after the join to validate your data.